In [ ]:
# Mapa da eleição de Dep. Estadual de 2018 por Zona Eleitoral nos distritos de SP

In [ ]:
'''
Fontes:
TSE - zonas eleitorais SP e resultado eleição
- votacao_candidato_munzona_2018_SP.csv:
http://agencia.tse.jus.br/estatistica/sead/odsele/votacao_candidato_munzona/votacao_candidato_munzona_2018.zip
- lista_zonas_eleitorais.csv:
http://www.tse.jus.br/eleitor/cartorios-e-zonas-eleitorais/pesquisa-a-zonas-eleitorais

Estadão Dados - shapefile
- zonaseleitorais2.shp:
https://groups.google.com/forum/#!topic/thackday/oJ-DRcsJL3w
'''

In [1]:
%matplotlib inline
import pandas as pd
import geopandas as gpd

In [2]:
cols = ['CD_MUNICIPIO', 'NM_MUNICIPIO', 'NR_ZONA', 'DS_CARGO',
        'NR_CANDIDATO', 'NM_URNA_CANDIDATO', 'NR_PARTIDO', 'SG_PARTIDO', 
        'QT_VOTOS_NOMINAIS']

df_chunk = pd.read_csv('./votos/votacao_candidato_munzona_2018_SP.csv', sep=';', usecols=cols, encoding='latin1', chunksize=10000)

In [3]:
DS_CARGO = 'Deputado Estadual'

In [4]:
chunk_list = []

for chunk in df_chunk:  
    chunk_filter = chunk[chunk['DS_CARGO'] == DS_CARGO]
    chunk_list.append(chunk_filter)    
df_concat = pd.concat(chunk_list)

In [16]:
# Agrupar por zona e candidato
df_zona = df_concat.groupby(['NR_ZONA', 'NR_CANDIDATO', 'NM_URNA_CANDIDATO', 'NR_PARTIDO', 'SG_PARTIDO'])['QT_VOTOS_NOMINAIS'].sum().reset_index()
df_zona.set_index('NR_ZONA', drop=False, inplace=True)

In [17]:
# Obter lista de zonas eleitorais do estado de São Paulo
df = pd.read_csv('./votos/lista_zonas_eleitorais.csv', sep=',', encoding='latin1')

In [18]:
# Filtrar somente pela cidade de São Paulo
df_sp = df[df['nome_municipio'] == 'SÃO PAULO']
df_sp.set_index('numero_zona', drop=False, inplace=True)
df_sp.shape

(58, 7)

In [19]:
df_zona_sp = df_zona.join(df_sp)
df_zona_sp.dropna(inplace=True)

In [20]:
# Definir candidato para analisar
NR_CANDIDATO = 17317

In [39]:
df_zona_sp_cand = df_zona_sp[df_zona_sp['NR_CANDIDATO'] == NR_CANDIDATO]
df_zona_sp_cand.shape
df_zona_sp_cand.head()

,NR_ZONA,NR_CANDIDATO,NM_URNA_CANDIDATO,NR_PARTIDO,SG_PARTIDO,QT_VOTOS_NOMINAIS,numero_zona,sigla_uf,endereco,cep,bairro,nome_municipio,sigla_uf.1
1,1,17317,JANAINA PASCHOAL,17,PSL,12117,1.0,26-0001,"AVENIDA BRIGADEIRO LUÍS ANTONIO, 453",1317000.0,BELA VISTA,SÃO PAULO,SP
2,2,17317,JANAINA PASCHOAL,17,PSL,18932,2.0,26-0002,"RUA DR. COSTA JÚNIOR, 509",5002000.0,AGUA BRANCA,SÃO PAULO,SP
3,3,17317,JANAINA PASCHOAL,17,PSL,4970,3.0,26-0003,"RUA ANTONIO CORUJA, 99/109",1126030.0,BOM RETIRO,SÃO PAULO,SP
4,4,17317,JANAINA PASCHOAL,17,PSL,20264,4.0,26-0004,"RUA MADRE DE DEUS, 427",3119000.0,MOOCA,SÃO PAULO,SP
5,5,17317,JANAINA PASCHOAL,17,PSL,16191,5.0,26-0005,"RUA CLODOMIRO AMAZONAS, 41",4537010.0,ITAIM BIBI,SÃO PAULO,SP


In [40]:
# Obter shapefile da cidade por zonas eleitorais
df_mapa = gpd.read_file('./geo/Zonas_eleitorais_SP_shp/zonaseleitorais2.shp')
df_mapa['Name'] = df_mapa['Name'].astype(int)
df_mapa.sort_values(by='Name', ascending=True, inplace=True)
df_mapa.set_index('Name', drop=False, inplace=True)

In [41]:
df_mapa_cand = df_mapa.join(df_zona_sp_cand)
df_mapa_cand.sort_values('QT_VOTOS_NOMINAIS', ascending=False, inplace=True)

In [42]:
import json
from bokeh.io import output_notebook, show, output_file
from bokeh.models import LogColorMapper, LinearColorMapper, ColorBar, LogTicker, GeoJSONDataSource
from bokeh.palettes import Viridis6 as palette
from bokeh.palettes import brewer
from bokeh.plotting import figure

In [43]:
geosource = GeoJSONDataSource(geojson = df_mapa_cand.to_json())

In [44]:
color_mapper = LinearColorMapper(palette=palette, low = 0, high = df_mapa_cand['QT_VOTOS_NOMINAIS'].iloc[0])

TOOLS = "pan,wheel_zoom,reset,hover,save"

color_bar = ColorBar(color_mapper=color_mapper,
                     label_standoff=12,
                     #width = 500,
                     #height = 20,
                     border_line_color=None,
                     location = (0,0), 
                     orientation = 'horizontal')

p = figure(
    title="Eleição de Dep. Estadual de 2018 por Distrito: " + df_mapa_cand['NM_URNA_CANDIDATO'].iloc[0], 
    tools=TOOLS,
    plot_height = 920,
    plot_width = 680,
    x_axis_location=None,
    y_axis_location=None,
    tooltips=[
        ("No da Zona", "@NR_ZONA"), ("Zona Eleitoral", "@bairro"), ("Votos", "@QT_VOTOS_NOMINAIS"), ("Partido", "@SG_PARTIDO")
    ])
p.grid.grid_line_color = None
p.hover.point_policy = "follow_mouse"

p.patches('xs','ys', source=geosource,
          fill_color={'field': 'QT_VOTOS_NOMINAIS', 'transform': color_mapper},
          fill_alpha=1.0, line_color="gray", line_width=0.5)

p.add_layout(color_bar, 'below')

output_notebook()

show(p)

Loading BokehJS ...

In [60]:
# Definir partido para analisar
NR_PARTIDO = 13

In [61]:
df_zona_sp_part = df_zona_sp[df_zona_sp['NR_PARTIDO'] == NR_PARTIDO]
df_zona_sp_part = df_zona_sp_part.groupby(['NR_ZONA','SG_PARTIDO', 'bairro'])['QT_VOTOS_NOMINAIS'].sum().reset_index()
df_zona_sp_part.set_index('NR_ZONA', drop=False, inplace=True)

In [62]:
df_mapa_part = df_mapa.join(df_zona_sp_part)
df_mapa_part.sort_values('QT_VOTOS_NOMINAIS', ascending=False, inplace=True)

In [63]:
geosource_part = GeoJSONDataSource(geojson = df_mapa_part.to_json())

In [64]:
color_mapper = LinearColorMapper(palette=palette, low = 0, high = df_mapa_part['QT_VOTOS_NOMINAIS'].iloc[0])

TOOLS = "pan,wheel_zoom,reset,hover,save"

color_bar = ColorBar(color_mapper=color_mapper,
                     label_standoff=12,
                     #width = 500,
                     #height = 20,
                     border_line_color=None,
                     location = (0,0), 
                     orientation = 'horizontal')

p = figure(
    title="Eleição de Dep. Estaduais de 2018 por Distrito - Partido: " + df_mapa_part['SG_PARTIDO'].iloc[0], 
    tools=TOOLS,
    plot_height = 920,
    plot_width = 680,
    x_axis_location=None,
    y_axis_location=None,
    tooltips=[
        ("No da Zona", "@NR_ZONA"), ("Zona Eleitoral", "@bairro"), ("Votos", "@QT_VOTOS_NOMINAIS"), ("Partido", "@SG_PARTIDO")
    ])
p.grid.grid_line_color = None
p.hover.point_policy = "follow_mouse"

p.patches('xs','ys', source=geosource_part,
          fill_color={'field': 'QT_VOTOS_NOMINAIS', 'transform': color_mapper},
          fill_alpha=1.0, line_color="gray", line_width=0.5)

p.add_layout(color_bar, 'below')

output_notebook()

show(p)

Loading BokehJS ...